In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Concatenate, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [2]:
df = pd.read_csv('../dataset/dataset.csv')

In [3]:
train_data, test_data = train_test_split(df, test_size=0.2)
input_features = ['LTP', 'timestamp'] 
target_variable = 'Symbol'

In [4]:
label_encoder = LabelEncoder()
train_data[target_variable] = label_encoder.fit_transform(train_data[target_variable])

In [ ]:
scaler = MinMaxScaler()
train_data[input_features] = scaler.fit_transform(train_data[input_features])


In [ ]:
def transformer_encoder(inputs, num_heads, hidden_units, dropout_rate):
    # MHSA bolte
    attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=hidden_units)(inputs, inputs)
    attention = tf.keras.layers.Dropout(dropout_rate)(attention)
    attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention)
    
    # FFN bolte
    outputs = tf.keras.layers.Dense(hidden_units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(inputs.shape[-1])(outputs)
    outputs = tf.keras.layers.Dropout(dropout_rate)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention + outputs)
    
    return outputs

In [ ]:
def transformer_model(input_dim, output_dim, num_heads, hidden_units, num_blocks, dropout_rate):
    inputs = Input(shape=(input_dim,))
    x = Embedding(input_dim, output_dim)(inputs)
    
    time_inputs = Input(shape=(1,))
    time_embed = Embedding(1440, output_dim)(time_inputs)  
    x = Concatenate()([x, time_embed])
    
    for _ in range(num_blocks):
        x = transformer_encoder(x, num_heads, hidden_units, dropout_rate)
    
    x = Flatten()(x)
    outputs = Dense(output_dim, activation='softmax')(x)
    
    model = Model(inputs=[inputs, time_inputs], outputs=outputs)
    return model


In [ ]:
input_dim = len(input_features)
output_dim = 64
num_heads = 8
hidden_units = 256
num_blocks = 4
dropout_rate = 0.1
batch_size = 32
epochs = 10

In [ ]:
model = transformer_model(input_dim, output_dim, num_heads, hidden_units, num_blocks, dropout_rate)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
train_input = [train_data[input_features].values, train_data['timestamp'].values]
train_target = train_data[target_variable].values


In [ ]:
model.fit(train_input, train_target, batch_size=batch_size, epochs=epochs)


In [ ]:
model.save('fno_predictoni.h5')